# Minecraft Texture & Skin AI Generator
Скрипт для сбора текстур Minecraft для датасета

In [ ]:
# 1. Установка зависимостей
!pip install beautifulsoup4 Pillow requests

In [ ]:
# 2. Создаем create_folders.py
%%writefile create_folders.py
import os

def create_dataset_structure():
    """
    Создает структуру папок для датасета
    """
    folders = [
        # Текстуры
        "dataset/textures/blocks/natural",
        "dataset/textures/blocks/ores",
        "dataset/textures/blocks/vegetation",
        "dataset/textures/blocks/building",
        "dataset/textures/special/gui",
        "dataset/textures/special/particles",
        "dataset/textures/special/animated",
        # Скины
        "dataset/skins/classic",
        "dataset/skins/fantasy",
        "dataset/skins/modern",
        "dataset/skins/historical",
        # Обработанные данные
        "dataset/processed/16x16",
        "dataset/processed/32x32",
        "dataset/processed/64x64"
    ]
    
    for folder in folders:
        os.makedirs(folder, exist_ok=True)
        print(f"Created: {folder}")

if __name__ == "__main__":
    create_dataset_structure()

In [ ]:
# 3. Создаем data_collector.py
%%writefile data_collector.py
# [Здесь весь код data_collector.py]

In [ ]:
# 4. Создаем test_texture_download.py
%%writefile test_texture_download.py
# [Здесь весь код test_texture_download.py]

In [ ]:
# 5. Запускаем тест
!python test_texture_download.py

In [ ]:
# 6. Проверяем структуру папок
!tree dataset/textures